### Converting Model to mobile-compatible state

In [1]:
import torch
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile

In [2]:
class EfficientNet_model(nn.Module):
    def __init__(self, num_classes, hidden_size=256, num_layers=2, dropout=0.25):
        super().__init__()
        # Load pre-trained model
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        # Freeze parameters of the pre-trained model
        for param in self.efficientnet.parameters():
            param.requires_grad = False
        # Temporal feature extraction through BI-LSTM
        self.lstm = nn.LSTM(input_size=1280, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.relu = nn.ReLU()
        # Dropout to prevent overfitting
        self.dropout1 = nn.Dropout(dropout)
        # Fully connected layer
        self.fc1 = nn.Linear(hidden_size * 2, 256)
        # Dropout to prevent overfitting
        self.dropout2 = nn.Dropout(dropout)
        self.relu2 = nn.Relu()
        # Last fully connected layer with number of classes as units to make predictions
        self.fc2 = nn.Linear(256, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.efficientnet.extract_features(x)
        # Flatten the feature maps and add a batch dimension
        x = x.flatten(start_dim=2).permute(0, 2, 1)
        # Pass the flattened feature maps through the bidirectional LSTM layer
        x, _ = self.lstm(x)
        x = self.relu(x)
        # Concatenate the outputs of the forward and backward directions and pass through the fully connected layers with dropout
        x = self.dropout1(torch.cat((x[:, -1, :self.lstm.hidden_size], x[:, 0, self.lstm.hidden_size:]), dim=1))
        x = self.fc1(x)
        x = self.dropout2(x)
        x = self.relu2(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


In [3]:
model = torch.load('C:/Users/Bruss/Desktop/Speciale/models/deployments/finalmodel_efficientnet_nolandmarks.pt')
model.eval()
model.efficientnet.set_swish(False)
device = torch.device('cpu')
model.to(device)
print(' ')

In [6]:
scripted_module = torch.jit.trace(model, torch.rand(1,3,224,224))

In [7]:
#scripted_module = torch.jit.script(model)
scripted_module.save("effecientnet_scripted.pt")

In [8]:
traced_script_optimized = optimize_for_mobile(scripted_module)